In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.linear_model import ElasticNetCV, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer


pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

make linear regression model with HSCs only

for age variable, make 2 separate models
- clean `age`, convert it to cts variable
- one hot encode `narrow_age_range`

In [2]:
cell_GEP_mapping = pd.read_csv(r'./data/Factorized matrices from human lifetime scRNA(GEP usage per cell).csv').set_index('Cell')

In [3]:
adata = sc.read_h5ad("./data/seurat_object.h5ad")

# Extract the expression matrix (cells x genes)
expression_df = pd.DataFrame(
    adata.X, 
    index=adata.obs.index, 
    columns=adata.var.index
)

# Extract cell metadata
cell_metadata_df = adata.obs

# Extract gene metadata
gene_metadata_df = adata.var

In [4]:
df = cell_GEP_mapping.copy()
merged = df.merge(cell_metadata_df, left_index=True, right_index=True)
df = merged[merged['subcluster'] == 'HSC'].drop(['sample', 'cluster', 'subcluster', 'orig.ident', 'broad_age_range'], axis=1)
df.head()

,GEP 1 (Mono/DC),GEP 2 (single donor-specific),GEP 3,GEP 4 (Elderly-HSC),GEP 5,GEP 6,GEP 7,GEP 8,GEP 9,GEP 10,GEP 11,GEP 12,GEP 13,GEP 14 (Meg),GEP 15 (DNA Replication),GEP 16,GEP 17,GEP 18,GEP 19,GEP 20,GEP 21 (Mitochondiral genes),GEP 22 (Cell cycle),GEP 23,GEP 24,GEP 25 (Baso/Mast),GEP 26 (Lymphoid),GEP 27 (Ribosomal biogenesis),GEP 28 (Fetal-HSC),GEP 29,GEP 30 (Granulocyte),GEP 31,GEP 32 (Erythroid),GEP 33 (Pan-cellular expression),GEP 34,GEP 35,nCount_RNA,nFeature_RNA,age,narrow_age_range
BM1_bcCHLB,0.010666,0.000000,0.0,0.435595,0.000000,0.000000,0.020563,0.011753,0.016536,0.017280,0.000000,0.033893,0.000681,0.029359,0.070330,0.0,0.000000,0.000000,0.000000,0.000000,0.041981,0.0,0.0065,0.003607,0.000000,0.000000,0.043055,0.0,0.000000,0.002227,0.016027,0.018264,0.221683,0.000000,0.000000,5050.0,2116,25yr-1,Adult
BM1_bcGZEW,0.045451,0.031609,0.0,0.517612,0.000000,0.000000,0.000000,0.009147,0.031807,0.000000,0.006016,0.000000,0.000000,0.016127,0.083941,0.0,0.000000,0.000000,0.000000,0.000000,0.036858,0.0,0.0000,0.000000,0.000030,0.004162,0.113646,0.0,0.000000,0.000000,0.000000,0.043661,0.000000,0.000000,0.059932,4119.0,1754,25yr-1,Adult
BM1_bcBLXF,0.095787,0.005552,0.0,0.568575,0.000139,0.000000,0.025774,0.019996,0.001473,0.000585,0.000000,0.000000,0.000000,0.007506,0.033607,0.0,0.024770,0.000000,0.001586,0.000054,0.065993,0.0,0.0000,0.000000,0.000000,0.019419,0.103474,0.0,0.000000,0.025711,0.000000,0.000000,0.000000,0.000000,0.000000,3334.0,1551,25yr-1,Adult
BM1_bcBDVL,0.013017,0.057985,0.0,0.532754,0.000000,0.000000,0.000000,0.000000,0.017790,0.000000,0.000000,0.011234,0.000000,0.002340,0.068884,0.0,0.000000,0.003215,0.017649,0.000000,0.038622,0.0,0.0000,0.000000,0.000000,0.015097,0.101843,0.0,0.009128,0.000000,0.000000,0.000000,0.036272,0.058949,0.015223,3355.0,1606,25yr-1,Adult
BM1_bcCMUW,0.037758,0.000000,0.0,0.462137,0.005303,0.000124,0.000000,0.000000,0.000000,0.000000,0.008137,0.000000,0.000116,0.000000,0.120970,0.0,0.006418,0.000627,0.000000,0.000000,0.039438,0.0,0.0000,0.000000,0.004656,0.010474,0.142781,0.0,0.005572,0.005173,0.004310,0.000000,0.146005,0.000000,0.000000,3044.0,1457,25yr-1,Adult


# model with `age` treated as cts variable, cleaned

In [5]:
def clean_age(ser):
    def clean_ser(age):
        if 'yr' in age:
            return float(age.split('yr')[0].strip())
        if 'CB' in age:
            return 0
        if 'wk' in age:
            return (float(age.split('wk')[0].strip()) - 40) / 52
        return None  # Handle unexpected cases
    
    # Apply `clean_ser` element-wise
    return ser.apply(clean_ser) if isinstance(ser, pd.Series) else np.vectorize(clean_ser)(ser)


In [6]:
eda = df.assign(cleaned_age=df['age'].apply(clean_age))

In [7]:
px.scatter(x=eda['cleaned_age'], y=eda['GEP 15 (DNA Replication)'])

In [8]:
grouped = eda.groupby('cleaned_age')['GEP 15 (DNA Replication)'].mean().to_frame().reset_index()
grouped

,cleaned_age,GEP 15 (DNA Replication)
0,-0.576923,0.069866
1,-0.557692,0.349863
2,-0.519231,0.072931
3,-0.500000,0.173315
4,-0.480769,0.182526
5,-0.384615,0.052060
6,-0.346154,0.278996
7,-0.326923,0.242951
8,0.000000,0.054846
9,2.000000,0.106414


In [9]:
px.line(x=grouped['cleaned_age'], y=grouped['GEP 15 (DNA Replication)'])

In [10]:
px.line(x=grouped['cleaned_age'].apply(np.log), y=grouped['GEP 15 (DNA Replication)'])

suggests there is a logarithmic relation between age and GEP 15 expression

In [11]:
def get_X_cts():
    return df.drop(['narrow_age_range', 'GEP 15 (DNA Replication)'], axis=1)

def get_y():
    return df['GEP 15 (DNA Replication)']

In [12]:
X_train_cts, X_test_cts, y_train_cts, y_test_cts = train_test_split(get_X_cts(), get_y(), test_size=0.2)

In [13]:
X_train_cts.head(2)

,GEP 1 (Mono/DC),GEP 2 (single donor-specific),GEP 3,GEP 4 (Elderly-HSC),GEP 5,GEP 6,GEP 7,GEP 8,GEP 9,GEP 10,GEP 11,GEP 12,GEP 13,GEP 14 (Meg),GEP 16,GEP 17,GEP 18,GEP 19,GEP 20,GEP 21 (Mitochondiral genes),GEP 22 (Cell cycle),GEP 23,GEP 24,GEP 25 (Baso/Mast),GEP 26 (Lymphoid),GEP 27 (Ribosomal biogenesis),GEP 28 (Fetal-HSC),GEP 29,GEP 30 (Granulocyte),GEP 31,GEP 32 (Erythroid),GEP 33 (Pan-cellular expression),GEP 34,GEP 35,nCount_RNA,nFeature_RNA,age
CB1_bcGDQI,0.064274,0.075202,0.0,0.243857,0.001776,0.000297,0.015160,0.0,0.021027,0.007091,0.004876,0.0,0.0,0.021626,0.000000,0.002915,0.0,0.000000,0.001614,0.023483,0.000000,0.000067,0.000705,0.0,0.0,0.169995,0.028313,0.0,0.056232,0.064319,0.0,0.156455,0.0,0.0,2776.0,1418,CB-1
GR74-bcERDQ,0.010834,0.000000,0.0,0.558866,0.000000,0.000000,0.018306,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.033194,0.000000,0.0,0.001382,0.000000,0.037949,0.010803,0.054520,0.000000,0.0,0.0,0.000000,0.036013,0.0,0.015415,0.000000,0.0,0.000000,0.0,0.0,1741.0,899,23wk-2


In [14]:
preproc_cts = make_column_transformer(
    (FunctionTransformer(clean_age, validate=False), ['age']),
    remainder='passthrough'
)

model_cts = make_pipeline(
    preproc_cts,
    LinearRegression()
)

model_cts.fit(X_train_cts, y_train_cts)

c:\Users\suchi\anaconda3\envs\hsc\lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning:


The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).




Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('functiontransformer',
                                                  FunctionTransformer(func=<function clean_age at 0x000001F4A4148DC0>),
                                                  ['age'])])),
                ('linearregression', LinearRegression())])

In [15]:
print(f'base linreg HSC only age as cts variable R^2: {model_cts.score(X_test_cts, y_test_cts)}')

base linreg HSC only age as cts variable R^2: 0.9999999999999999


In [16]:
age_cts = pd.DataFrame({
    'Feature': X_train_cts.columns,
    'Weight': model_cts.named_steps['linearregression'].coef_
})
age_cts = pd.concat(
    [age_cts, pd.DataFrame({'Feature': ['Intercept'], 'Weight': [model_cts.named_steps['linearregression'].intercept_]})]
)

age_cts

,Feature,Weight
0,GEP 1 (Mono/DC),5.162583e-13
1,GEP 2 (single donor-specific),-1.000000e+00
2,GEP 3,-1.000000e+00
3,GEP 4 (Elderly-HSC),-1.000000e+00
4,GEP 5,-1.000000e+00
5,GEP 6,-1.000000e+00
6,GEP 7,-1.000000e+00
7,GEP 8,-1.000000e+00
8,GEP 9,-1.000000e+00
9,GEP 10,-1.000000e+00


# try running linear regression on HSC only with only other GEPs as features

In [17]:
def drop_non_GEP_features(X):
    return X.drop(columns=['nCount_RNA', 'nFeature_RNA', 'age'])

In [18]:
GEP_only_model = make_pipeline(
    FunctionTransformer(drop_non_GEP_features),
    LinearRegression()
)

GEP_only_model.fit(X_train_cts, y_train_cts)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function drop_non_GEP_features at 0x000001F4AA6F15A0>)),
                ('linearregression', LinearRegression())])

In [19]:
print(f'base linreg HSC gep as features only R^2: {GEP_only_model.score(X_test_cts, y_test_cts)}')

base linreg HSC gep as features only R^2: 0.9999999999999999


In [20]:
GEP_only_weights = pd.DataFrame({
    'Feature': X_train_cts.columns[:-3],
    'Weight': GEP_only_model.named_steps['linearregression'].coef_
})
GEP_only_weights = pd.concat(
    [GEP_only_weights, pd.DataFrame({'Feature': ['Intercept'], 'Weight': [model_cts.named_steps['linearregression'].intercept_]})]
)

GEP_only_weights

,Feature,Weight
0,GEP 1 (Mono/DC),-1.0
1,GEP 2 (single donor-specific),-1.0
2,GEP 3,-1.0
3,GEP 4 (Elderly-HSC),-1.0
4,GEP 5,-1.0
5,GEP 6,-1.0
6,GEP 7,-1.0
7,GEP 8,-1.0
8,GEP 9,-1.0
9,GEP 10,-1.0
